#### 목표 설정
- kiwipiepy 형태소 분석기를 사용해보고 Konlpy와는 어떤점이 다른지 확인

#### Kiwi 라이브러리
- 빠른속도와 범용적인 성능을 지향하는 형태소 분석기
- C++로 구현된 코어를 래핑해서 다양한 프로그래밍 언어에서 사용하도록 만들었다. / 오픈소스 공개
- 세종 품사 태그셋을 사용한다.
- 웹 텍스트 약 87% / 문어체 텍스튼느 94% 정확도로 형태소 분석이 가능하다. ( 공식 GitHub 참조 )
- Konlpy보다 속도가 더 빠르다.
- 간단한 오타나 띄어쓰기는 스스로 교정 할 수 있는 기능을 제공, 신조어나 오타에 강

In [91]:
# 경로 확인
import os
print("현재 작업 경로:", os.getcwd())

# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 불러오기 - WSL 환경에 맞게 경로 수정
# 데이터 파일은 ~/projects/data 폴더에 있다고 가정
train = pd.read_csv('../data/ratings_train.csv', encoding='utf-8')
test = pd.read_csv('../data/ratings_test.csv', encoding='utf-8')

# 데이터 확인
print("학습 데이터 미리보기:")
print(train.head())

print("\n테스트 데이터 미리보기:")
print(test.head())

현재 작업 경로: /home/jsock/core-project-study/jinseok/textmining/projects
학습 데이터 미리보기:
         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1

테스트 데이터 미리보기:
        id                                           document  label
0  6270596                                                굳 ㅋ      1
1  9274899                               GDNTOPCLASSINTHECLUB      0
2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0


In [92]:
train = train.dropna()
test = test.dropna()

In [93]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [94]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [95]:
X_train = train['document']
X_test = test['document']
y_train = train['label']
y_test = test['label']

In [96]:
# 데이터 shape 확인
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((149995,), (49997,), (149995,), (49997,))

Kiwipiepy 라이브러리 설치
- Kiwipiepy 라이브러리는 의존성이 없음. 편리하게 환경 구축이 가능하다.

In [97]:
pip install kiwipiepy

Note: you may need to restart the kernel to use updated packages.


In [98]:
# 라이브러리 import
# 사이킷런 Tfidf Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Kiwipiepy 형태소 분석기 import
from kiwipiepy import Kiwi, basic_typos_with_continual

In [99]:
# 객체 생성
kiwi = Kiwi(typos = basic_typos_with_continual)
kiwi

Kiwi(num_workers=1, model_path=None, integrate_allomorph=True, load_default_dict=True, load_typo_dict=True, model_type='knlm', typos=TypoTransformer([('\x00가', '까', 1.0, 'applosive'), ('\x00개', '깨', 1.0, 'applosive'), ('\x00갸', '꺄', 1.0, 'applosive'), ('\x00걔', '꺠', 1.0, 'applosive'), ('\x00거', '꺼', 1.0, 'applosive'), ... (11241 more)], continual_typo_cost=1.0, lengthening_typo_cost=inf), typo_cost_threshold=2.5)

- Kiwipiepy는 간단한 오타 교정도구를 제공
  - basic_typos : 형태소 내에 오타를 교정하는 기본적인 오타 교정 도구
  - continual_typos : 형태소간의 연철 오타를 교정하는 교정 도구
  - basic_typos_with_continual : 위의 두가지 오타 교정도구를 하나로 합친 기능

In [100]:
# 형태소 분석 - tokenize
token = kiwi.tokenize('아버지가방에들어가신다')
token

# form : 실제 형태소
# tag : 품사 태그
# start : 형태소의 시작 인덱스
# len : 형태소의 길

[Token(form='아버지', tag='NNG', start=0, len=3),
 Token(form='가', tag='JKS', start=3, len=1),
 Token(form='방', tag='NNG', start=4, len=1),
 Token(form='에', tag='JKB', start=5, len=1),
 Token(form='들어가', tag='VV', start=6, len=3),
 Token(form='시', tag='EP', start=9, len=1),
 Token(form='ᆫ다', tag='EF', start=9, len=2)]

In [101]:
token[0].tag

'NNG'

In [102]:
# 형태소 분석된 곳에서 실제 형태소만 가져오자
text = '오늘은 수요일입니당. 금요일이 오기는 올까욬ㅋㅋㅋㅋ'

# 형태소 분석 시작
token = kiwi.tokenize(text)
token

[Token(form='오늘', tag='NNG', start=0, len=2),
 Token(form='은', tag='JX', start=2, len=1),
 Token(form='수요일', tag='NNG', start=4, len=3),
 Token(form='이', tag='VCP', start=7, len=1),
 Token(form='ᆸ니다', tag='EF', start=7, len=3),
 Token(form='ᆼ', tag='Z_CODA', start=9, len=1),
 Token(form='.', tag='SF', start=10, len=1),
 Token(form='금요일', tag='NNG', start=12, len=3),
 Token(form='이', tag='JKS', start=15, len=1),
 Token(form='오', tag='VV', start=17, len=1),
 Token(form='기', tag='ETN', start=18, len=1),
 Token(form='는', tag='JX', start=19, len=1),
 Token(form='오', tag='VV', start=21, len=1),
 Token(form='ᆯ까요', tag='EF', start=21, len=3),
 Token(form='ᆿ', tag='Z_CODA', start=23, len=1),
 Token(form='ㅋㅋㅋㅋ', tag='SW', start=24, len=4)]

In [103]:
# 오타 교정도구 옵션 수정하기
from kiwipiepy import basic_typos

kiwi2 = Kiwi(typos=basic_typos)

In [104]:
# '욬'에 있는 'ㅋ' wpdhl

# 더미 문장 데이터를 이용해서, kiwi 라이브러리 사용법 익혀보기
# 형태소 분석 시작

# nomalize_coda
# 초성체가 어절 뒤에 붙는 경우, 초성체를 분리해주는 기능
kiwi2 = Kiwi()

text = '오늘은 수요일입니당. 금요일이 오기는 올까욬ㅋㅋㅋㅋ'

# tokenize()에만 normalize_coda=True 옵션을 전달합니다 (스펠링 주의!)
token = kiwi2.tokenize(text, normalize_coda=True)
token

[Token(form='오늘', tag='NNG', start=0, len=2),
 Token(form='은', tag='JX', start=2, len=1),
 Token(form='수요일', tag='NNG', start=4, len=3),
 Token(form='이', tag='VCP', start=7, len=1),
 Token(form='ᆸ니다', tag='EF', start=7, len=3),
 Token(form='ᆼ', tag='Z_CODA', start=9, len=1),
 Token(form='.', tag='SF', start=10, len=1),
 Token(form='금요일', tag='NNG', start=12, len=3),
 Token(form='이', tag='JKS', start=15, len=1),
 Token(form='오', tag='VV', start=17, len=1),
 Token(form='기', tag='ETN', start=18, len=1),
 Token(form='는', tag='JX', start=19, len=1),
 Token(form='오', tag='VV', start=21, len=1),
 Token(form='ᆯ까요', tag='EF', start=21, len=3),
 Token(form='ㅋㅋㅋㅋㅋ', tag='SW', start=23, len=5)]

In [105]:
token2 = kiwi2.tokenize(text, normalize_coda=False)
token2

[Token(form='오늘', tag='NNG', start=0, len=2),
 Token(form='은', tag='JX', start=2, len=1),
 Token(form='수요일', tag='NNG', start=4, len=3),
 Token(form='이', tag='VCP', start=7, len=1),
 Token(form='ᆸ니다', tag='EF', start=7, len=3),
 Token(form='ᆼ', tag='Z_CODA', start=9, len=1),
 Token(form='.', tag='SF', start=10, len=1),
 Token(form='금요일', tag='NNG', start=12, len=3),
 Token(form='이', tag='JKS', start=15, len=1),
 Token(form='오', tag='VV', start=17, len=1),
 Token(form='기', tag='ETN', start=18, len=1),
 Token(form='는', tag='JX', start=19, len=1),
 Token(form='오', tag='VV', start=21, len=1),
 Token(form='ᆯ까요', tag='EF', start=21, len=3),
 Token(form='ᆿ', tag='Z_CODA', start=23, len=1),
 Token(form='ㅋㅋㅋㅋ', tag='SW', start=24, len=4)]

In [106]:
# split_complex
# 합성어를 분리하는 기능 - (기본값 : Fasle)
# 형태소 : 의미를 가진 가장 작은 언어의 단위
kiwi2.tokenize('얼마전 국수집에 방문했더니, 설탕통이 번쩍거리며 눈에 들어왔다.', split_complex = True)

# 형태소를 최대한 잘게 분할하는 기능

[Token(form='얼마', tag='NNG', start=0, len=2),
 Token(form='전', tag='NNG', start=2, len=1),
 Token(form='국수', tag='NNG', start=4, len=2),
 Token(form='집', tag='NNG', start=6, len=1),
 Token(form='에', tag='JKB', start=7, len=1),
 Token(form='방문', tag='NNG', start=9, len=2),
 Token(form='하', tag='XSV', start=11, len=1),
 Token(form='었', tag='EP', start=11, len=1),
 Token(form='더니', tag='EC', start=12, len=2),
 Token(form=',', tag='SP', start=14, len=1),
 Token(form='설탕', tag='NNG', start=16, len=2),
 Token(form='통', tag='NNG', start=18, len=1),
 Token(form='이', tag='JKS', start=19, len=1),
 Token(form='번쩍', tag='MAG', start=21, len=2),
 Token(form='거리', tag='XSV', start=23, len=2),
 Token(form='며', tag='EC', start=25, len=1),
 Token(form='눈', tag='NNG', start=27, len=1),
 Token(form='에', tag='JKB', start=28, len=1),
 Token(form='들', tag='VV', start=30, len=1),
 Token(form='어', tag='EC', start=31, len=1),
 Token(form='오', tag='VX', start=32, len=1),
 Token(form='었', tag='EP', start=32, len

In [107]:
kiwi.tokenize('외않됀대?')

[Token(form='왜', tag='MAG', start=0, len=1),
 Token(form='안', tag='MAG', start=1, len=1),
 Token(form='되', tag='VV', start=2, len=1),
 Token(form='ᆫ대', tag='EF', start=2, len=2),
 Token(form='?', tag='SF', start=4, len=1)]

In [108]:
# 위의 기능을 함수화 시켜서 토큰화 도구에 연결해보자
def myTokenizer(text):
    result = kiwi.tokenize(text)
    for token in result :
        if token.tag in ['NNG','NNP']: # 보통명사, 
            yield token.form

#### return이 아닌 yield를 쓰는 이유
- yeild : 반환값을 제너레이터로 변환
- 제너레이터 : 여러개의 데이터를 미리 만들어 두는 것이 아닌, 필요할때마다 즉석으로 하나씩 만들어주는 객체 -> for문에 사용 가능

In [109]:
# 함수 정의
def return_abc():
    return 'A'
    return 'B'
    return 'C'

def yield_abc():
    yield 'A'
    yield 'B'
    yield 'C'

In [110]:
print(return_abc())

A


In [111]:
print(yield_abc())

<generator object yield_abc at 0x7fea110c0930>


In [112]:
for ch in return_abc():
    print(ch)

A


In [113]:
import time

In [114]:
# 함수 정의시 반복분
def return_abc():
    for ch in 'ABC':
        time.sleep(1)
        return ch

In [115]:
for ch in return_abc():
    print(ch)

A


In [116]:
# 함수 정의시 반복분
def yield_abc():
    for ch in 'ABC':
        time.sleep(1)
        yield ch

In [117]:
for ch in yield_abc():
    print(ch)

A
B
C


In [118]:
# 더미문장 입력해보기
list(myTokenizer('어제는 커피를 마셨고, 오늘은 지코를 마실겁니다.'))


['어제', '커피', '오늘', '지코']

In [119]:
X_train[:3]

0                  아 더빙.. 진짜 짜증나네요 목소리
1    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                    너무재밓었다그래서보는것을추천한다
Name: document, dtype: object

In [120]:
# tfidf vectorizer + mytokenizer
tfidf = TfidfVectorizer(tokenizer = myTokenizer )

In [121]:
tfidf.fit(X_train[0:3])
tfidf.vocabulary_

/home/jsock/datascience_env/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'더빙': 1,
 '짜증': 7,
 '목소리': 2,
 '포스터': 10,
 '초딩': 8,
 '영화': 4,
 '줄': 6,
 '오버': 5,
 '연기': 3,
 '너무재밓': 0,
 '추천': 9}

In [122]:
# 감성분석을 위해 동사/형용사/보통명사만 가져오자
def myTokenizer2(text):
    # 띄어쓰기 교정
    # space
    text_space = kiwi.space(text, reset_whitespace = True)
    # reset_whitespace = False : 원래 문장들의 공백을 최대한 유지하면서, 필요한 부분만 수정
    # reset_whitespace = True : 원래 문장들의 공백정보를 무시하고, 새롭게 띄어쓰기 재구성

    result = kiwi.tokenize(text_space, split_complex=True)

    for token in result:
        if token.tag in ['NNG', 'VV', 'VA']:
            yield token.form

In [123]:
tfidf = TfidfVectorizer(tokenizer = myTokenizer2)
tfidf.fit(X_train[0:3])
tfidf.vocabulary_

{'더빙': 1,
 '짜증': 8,
 '나': 0,
 '목소리': 2,
 '포스터': 10,
 '보': 3,
 '영화': 5,
 '줄': 7,
 '오버': 6,
 '연기': 4,
 '추천': 9}

In [124]:
final_tfidf = TfidfVectorizer(tokenizer = myTokenizer2)
final_tfidf.fit(X_train[0:10000])

TfidfVectorizer(tokenizer=<function myTokenizer2 at 0x7fea100fa840>)

In [125]:
# 실제 데이터 토큰화 시작
# konly의 kkma보다 속도가 더 빠름
len(final_tfidf.vocabulary_)

6845

In [126]:
max(final_tfidf.vocabulary_.values())

6844

In [127]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [128]:
X_train_token = final_tfidf.transform(X_train[0:10000])
X_test_token = final_tfidf.transform(X_test[0:10000])

In [135]:
# fit_transform

# X_train[:10000] 데이터를 가지고 단어사전을 구축한 후 문장을 토큰화(위의 과정 한번에 진행)
# X_train_token = tfidf.fit_transform(X_train[0:10000])

# 주의점 : test데이터에서는 사용하지 않음.

### 모델링

In [137]:
from sklearn.linear_model import LogisticRegression

In [138]:
logi = LogisticRegression()

In [140]:
logi.fit(X_train_token, y_train[0:10000])

LogisticRegression()

In [142]:
# 훈련데이터 예측 정확도
logi.score(X_test_token, y_test[0:10000])

0.7714

In [143]:
pip install wordcloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.2/539.2 kB 16.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
